## 环境配置

安装MindSpore框架和MindNLP套件

In [1]:
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/unified/aarch64/mindspore-2.5.0-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/unified/aarch64/mindspore-2.5.0-cp39-cp39-linux_aarch64.whl (345.0 MB)
ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: No matching distribution found for install


In [2]:
!pip install mindnlp==0.4.0

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple/

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


#### 注：MindNLP whl包下载链接为：[MindNLP](https://repo.mindspore.cn/mindspore-lab/mindnlp/newest/any/)

# 基于 MindSpore 实现 BERT 对话情绪识别


## 模型简介

BERT全称是来自变换器的双向编码器表征量（Bidirectional Encoder Representations from Transformers），它是Google于2018年末开发并发布的一种新型语言模型。与BERT模型相似的预训练语言模型例如问答、命名实体识别、自然语言推理、文本分类等在许多自然语言处理任务中发挥着重要作用。模型是基于Transformer中的Encoder并加上双向的结构，因此一定要熟练掌握Transformer的Encoder的结构。

BERT模型的主要创新点都在pre-train方法上，即用了Masked Language Model和Next Sentence Prediction两种方法分别捕捉词语和句子级别的representation。

在用Masked Language Model方法训练BERT的时候，随机把语料库中15%的单词做Mask操作。对于这15%的单词做Mask操作分为三种情况：80%的单词直接用[Mask]替换、10%的单词直接替换成另一个新的单词、10%的单词保持不变。

因为涉及到Question Answering (QA) 和 Natural Language Inference (NLI)之类的任务，增加了Next Sentence Prediction预训练任务，目的是让模型理解两个句子之间的联系。与Masked Language Model任务相比，Next Sentence Prediction更简单些，训练的输入是句子A和B，B有一半的几率是A的下一句，输入这两个句子，BERT模型预测B是不是A的下一句。

BERT预训练之后，会保存它的Embedding table和12层Transformer权重（BERT-BASE）或24层Transformer权重（BERT-LARGE）。使用预训练好的BERT模型可以对下游任务进行Fine-tuning，比如：文本分类、相似度判断、阅读理解等。

对话情绪识别（Emotion Detection，简称EmoTect），专注于识别智能对话场景中用户的情绪，针对智能对话场景中的用户文本，自动判断该文本的情绪类别并给出相应的置信度，情绪类型分为积极、消极、中性。 对话情绪识别适用于聊天、客服等多个场景，能够帮助企业更好地把握对话质量、改善产品的用户交互体验，也能分析客服服务质量、降低人工质检成本。

下面以一个文本情感分类任务为例子来说明BERT模型的整个应用过程。

In [3]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn, context

from mindnlp.engine.trainer import Trainer

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the default dict

In [4]:
# prepare dataset
class SentimentDataset:
    """Sentiment Dataset"""

    def __init__(self, path):
        self.path = path
        self._labels, self._text_a = [], []
        self._load()

    def _load(self):
        with open(self.path, "r", encoding="utf-8") as f:
            dataset = f.read()
        lines = dataset.split("\n")
        for line in lines[1:-1]:
            label, text_a = line.split("\t")
            self._labels.append(int(label))
            self._text_a.append(text_a)

    def __getitem__(self, index):
        return self._labels[index], self._text_a[index]

    def __len__(self):
        return len(self._labels)

## 数据集

这里提供一份已标注的、经过分词预处理的机器人聊天数据集，来自于百度飞桨团队。数据由两列组成，以制表符（'\t'）分隔，第一列是情绪分类的类别（0表示消极；1表示中性；2表示积极），第二列是以空格分词的中文文本，如下示例，文件为 utf8 编码。

label--text_a

0--谁骂人了？我从来不骂人，我骂的都不是人，你是人吗 ？

1--我有事等会儿就回来和你聊

2--我见到你很高兴谢谢你帮我

这部分主要包括数据集读取，数据格式转换，数据 Tokenize 处理和 pad 操作。

In [5]:
# download dataset
!wget https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz -O emotion_detection.tar.gz
!tar xvf emotion_detection.tar.gz

--2025-06-03 16:26:40--  https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz
正在解析主机 baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)... 36.110.192.178, 2409:8c04:1001:1203:0:ff:b0bb:4f27
正在连接 baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)|36.110.192.178|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：1710581 (1.6M) [application/x-gzip]
正在保存至: “emotion_detection.tar.gz”

emotion_detection.t 100%[===================>]   1.63M  7.02MB/s  用时 0.2s      

2025-06-03 16:26:41 (7.02 MB/s) - 已保存 “emotion_detection.tar.gz” [1710581/1710581])

data/
data/test.tsv
data/infer.tsv
data/dev.tsv
data/train.tsv
data/vocab.txt


### 数据加载和数据预处理

新建 process_dataset 函数用于数据加载和数据预处理，具体内容可见下面代码注释。

In [6]:
import numpy as np

def process_dataset(source, tokenizer, max_seq_len=64, batch_size=32, shuffle=True):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'

    column_names = ["label", "text_a"]
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    type_cast_op = transforms.TypeCast(mindspore.int32)
    def tokenize_and_pad(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text)
        return tokenized['input_ids'], tokenized['attention_mask']
    # map dataset
    dataset = dataset.map(operations=tokenize_and_pad, input_columns="text_a", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=[type_cast_op], input_columns="label", output_columns='labels')
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})

    return dataset

昇腾NPU环境下暂不支持动态Shape，数据预处理部分采用静态Shape处理：

In [7]:
from mindnlp.transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

  0%|          | 0.00/49.0 [00:00<?, ?B/s]

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

  0%|          | 0.00/324 [00:00<?, ?B/s]

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [8]:
tokenizer.pad_token_id

0

In [9]:
dataset_train = process_dataset(SentimentDataset("data/train.tsv"), tokenizer)
dataset_val = process_dataset(SentimentDataset("data/dev.tsv"), tokenizer)
dataset_test = process_dataset(SentimentDataset("data/test.tsv"), tokenizer, shuffle=False)

In [10]:
dataset_train.get_col_names()

['input_ids', 'attention_mask', 'labels']

In [11]:
print(next(dataset_train.create_tuple_iterator()))

[Tensor(shape=[32, 64], dtype=Int64, value=
[[ 101, 1968, 1968 ...    0,    0,    0],
 [ 101,  679, 4761 ...    0,    0,    0],
 [ 101,  679, 3236 ...    0,    0,    0],
 ...
 [ 101, 7583, 7583 ...    0,    0,    0],
 [ 101,  872,  679 ...    0,    0,    0],
 [ 101, 2876, 2805 ...    0,    0,    0]]), Tensor(shape=[32, 64], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), Tensor(shape=[32], dtype=Int32, value= [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 
 1, 1, 1, 1, 2, 1, 0, 0])]


In [12]:
%env HF_ENDPOINT=https://hf-mirror.com

env: HF_ENDPOINT=https://hf-mirror.com


## 模型构建

通过 BertForSequenceClassification 构建用于情感分类的 BERT 模型，加载预训练权重，设置情感三分类的超参数自动构建模型。后面对模型采用自动混合精度操作，提高训练的速度，然后实例化优化器，紧接着实例化评价指标，设置模型训练的权重保存策略，最后就是构建训练器，模型开始训练。

In [13]:
from mindnlp.transformers import BertForSequenceClassification, BertModel

# set bert config and define parameters for training
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)

  0%|          | 0.00/392M [00:00<?, ?B/s]

[WARNING] DEVICE(3513,ffffa2651020,python):2025-06-03-16:41:44.906.806 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:147] CheckVmmDriverVersion] Open file /etc/ascend_install.info failed.
[WARNING] DEVICE(3513,ffffa2651020,python):2025-06-03-16:41:44.906.895 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:186] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.6
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import numpy as np

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    
    if len(predictions.shape) > 1:
        predictions = np.argmax(predictions, axis=-1)

    accuracy = (predictions == labels).mean()
    
    return {"accuracy": float(accuracy)}

In [15]:
from mindnlp.engine import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-5,
    num_train_epochs=5,
    logging_steps=200,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    args=training_args
)

In [16]:
# start training
trainer.train()

  0%|          | 0/1510 [00:00<?, ?it/s]

.{'loss': 0.3728, 'learning_rate': 1.7350993377483446e-05, 'epoch': 0.66}
.

  0%|          | 0/34 [00:00<?, ?it/s]

{'eval_loss': 0.17702096700668335, 'eval_accuracy': 0.9351851851851852, 'eval_runtime': 1.5801, 'eval_samples_per_second': 21.518, 'eval_steps_per_second': 21.518, 'epoch': 1.0}
{'loss': 0.2414, 'learning_rate': 1.4701986754966889e-05, 'epoch': 1.32}
{'loss': 0.18, 'learning_rate': 1.2052980132450332e-05, 'epoch': 1.99}


  0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/34 [00:00<?, ?it/s]

{'eval_loss': 0.056950367987155914, 'eval_accuracy': 0.9861111111111112, 'eval_runtime': 1.5702, 'eval_samples_per_second': 21.653, 'eval_steps_per_second': 21.653, 'epoch': 4.0}
{'loss': 0.0568, 'learning_rate': 1.456953642384106e-06, 'epoch': 4.64}


  0%|          | 0/34 [00:00<?, ?it/s]

{'eval_loss': 0.03606536239385605, 'eval_accuracy': 0.9916666666666667, 'eval_runtime': 1.8247, 'eval_samples_per_second': 18.633, 'eval_steps_per_second': 18.633, 'epoch': 5.0}
{'train_runtime': 666.5481, 'train_samples_per_second': 72.493, 'train_steps_per_second': 2.265, 'train_loss': 0.1572099215147511, 'epoch': 5.0}


TrainOutput(global_step=1510, training_loss=0.1572099215147511, metrics={'train_runtime': 666.5481, 'train_samples_per_second': 72.493, 'train_steps_per_second': 2.265, 'train_loss': 0.1572099215147511, 'epoch': 5.0})

## 模型验证

将验证数据集加再进训练好的模型，对数据集进行验证，查看模型在验证数据上面的效果，此处的评价指标为准确率。

In [17]:
from tqdm import tqdm
import numpy as np

def compute_accuracy(logits, labels):
    preds = np.argmax(logits, axis=-1)
    refs  = labels.asnumpy() if hasattr(labels, "asnumpy") else labels
    return {"accuracy": float((preds == refs).mean())}

def evaluate_fn(model, test_dataset):
    total = test_dataset.get_dataset_size()
    epoch_acc, step_total = 0.0, 0
    model.set_train(False)

    with tqdm(total=total) as progress_bar:
        for batch in test_dataset.create_dict_iterator():
            label  = batch.pop('labels')
            logits = model(**batch).logits

            acc_dict  = compute_accuracy(logits, label)
            epoch_acc += acc_dict['accuracy']

            step_total += 1
            progress_bar.update(1)
            progress_bar.set_postfix(acc=epoch_acc / step_total)

    return epoch_acc / step_total

acc = evaluate_fn(model, dataset_val)
print(f"Accuracy: {acc:.4f}")

100%|██████████| 34/34 [00:01<00:00, 18.97it/s, acc=0.992]


Accuracy: 0.9917


In [18]:
acc = evaluate_fn(model, dataset_val)
print(f"Accuracy: {acc}")

100%|██████████| 34/34 [00:01<00:00, 19.84it/s, acc=0.992]

Accuracy: 0.9917279411764706


## 模型推理

遍历推理数据集，将结果与标签进行统一展示。

In [19]:
dataset_infer = SentimentDataset("data/infer.tsv")

In [20]:
def predict(text, label=None):
    label_map = {0: "消极", 1: "中性", 2: "积极"}

    text_tokenized = Tensor([tokenizer(text).input_ids])
    logits = model(text_tokenized)
    predict_label = logits[0].asnumpy().argmax()
    info = f"inputs: '{text}', predict: '{label_map[predict_label]}'"
    if label is not None:
        info += f" , label: '{label_map[label]}'"
    print(info)

In [21]:
from mindspore import Tensor

for label, text in dataset_infer:
    predict(text, label)

inputs: '我 要 客观', predict: '中性' , label: '中性'
inputs: '靠 你 真是 说 废话 吗', predict: '消极' , label: '消极'
inputs: '口嗅 会', predict: '中性' , label: '中性'
inputs: '每次 是 表妹 带 窝 飞 因为 窝路痴', predict: '中性' , label: '中性'
inputs: '别说 废话 我 问 你 个 问题', predict: '消极' , label: '消极'
inputs: '4967 是 新加坡 那 家 银行', predict: '中性' , label: '中性'
inputs: '是 我 喜欢 兔子', predict: '积极' , label: '积极'
inputs: '你 写 过 黄山 奇石 吗', predict: '中性' , label: '中性'
inputs: '一个一个 慢慢来', predict: '中性' , label: '中性'
inputs: '我 玩 过 这个 一点 都 不 好玩', predict: '消极' , label: '消极'
inputs: '网上 开发 女孩 的 QQ', predict: '中性' , label: '中性'
inputs: '背 你 猜 对 了', predict: '中性' , label: '中性'
inputs: '我 讨厌 你 ， 哼哼 哼 。 。', predict: '消极' , label: '消极'


## 自定义推理数据集

自己输入推理数据，展示模型的泛化能力。

In [22]:
predict("家人们咱就是说一整个无语住了 绝绝子叠buff")

inputs: '家人们咱就是说一整个无语住了 绝绝子叠buff', predict: '中性'
